In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

import graphml.constants

In [2]:
constants = graphml.constants.Constants()

In [3]:
file = constants.DATA.joinpath('embed_cites_umap.csv.gz')
data = pd.read_csv(file)

/var/folders/0p/18prqvyd5kd_xr2h6whgkm600000gn/T/ipykernel_75942/2024754465.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file)


In [4]:
data.head()

,docId,title,category,embed[].0,embed[].1,embed[].2,embed[].3,embed[].4,embed[].5,embed[].6,...,umap_1,umap_2,umap_3,umap_4,umap_5,umap_6,umap_7,umap_8,umap_9,cluster_labels
0,704.0302,Spline Single-Index Prediction Model,stat.TH,0.000461,-0.001192,-0.003083,0.002297,-0.003558,-0.000685,-0.000624,...,5.031054,4.187748,3.858944,5.631053,4.896350,4.891944,5.386610,4.733177,4.506786,3
1,704.0326,On generalized entropy measures and pathways,stat.TH,0.000460,0.000133,0.001372,0.000597,-0.000666,-0.003240,0.003895,...,4.191865,3.565738,3.536421,5.208500,5.306760,4.914432,5.471216,5.011545,5.026620,3
2,704.0329,Solutions of fractional reaction-diffusion equ...,stat.TH,0.000457,-0.001057,0.000212,-0.000096,-0.001830,-0.000393,0.003056,...,4.361795,3.832480,4.358634,4.799599,4.438835,4.226096,5.579476,5.365624,4.456157,3
3,704.0517,Using decomposed household food acquisitions a...,stat.TH,0.000456,0.000268,-0.003145,-0.001797,0.001063,-0.002948,-0.000237,...,4.561526,3.591063,3.765388,4.994739,4.817182,5.429080,5.013913,5.303919,5.579634,3
4,704.0539,Integral representations for convolutions of n...,stat.TH,0.000458,-0.003706,-0.000887,0.003306,0.000198,-0.003094,0.001829,...,4.325488,4.465783,3.791301,4.735166,4.612527,4.527419,5.011487,4.712466,5.127670,3


In [30]:
node2vec_cols = [c for c in data.columns if 'embed[]' in c]
umap_cols = [c for c in data.columns if 'umap' in c]

## Using 128 Dimensions from Node2Vec

In [56]:
rec = NearestNeighbors(n_neighbors=20)
rec.fit(data.loc[:, node2vec_cols])

NearestNeighbors(n_neighbors=20)

In [57]:
paper = data.sample(1)
paper

,docId,title,category,embed[].0,embed[].1,embed[].2,embed[].3,embed[].4,embed[].5,embed[].6,...,umap_1,umap_2,umap_3,umap_4,umap_5,umap_6,umap_7,umap_8,umap_9,cluster_labels
49798,1808.0726,On an improvement of LASSO by scaling,stat.ML,0.002325,-0.001465,-0.00217,0.001215,-0.002538,0.002546,-0.000273,...,4.031207,3.787616,3.609304,5.304809,4.780702,5.380967,5.013962,4.745392,4.801008,3


**Title**: On an improvement of LASSO by scaling

**Abstract**: A sparse modeling is a major topic in machine learning and statistics. LASSO (Least Absolute Shrinkage and Selection Operator) is a popular sparse modeling method while it has been known to yield unexpected large bias especially at a sparse representation. There have been several studies for improving this problem such as the introduction of non-convex regularization terms. The important point is that this bias problem directly affects model selection in applications since a sparse representation cannot be selected by a prediction error based model selection even if it is a good representation. In this article, we considered to improve this problem by introducing a scaling that expands LASSO estimator to compensate excessive shrinkage, thus a large bias in LASSO estimator. We here gave an empirical value for the amount of scaling. There are two advantages of this scaling method as follows. Since the proposed scaling value is calculated by using LASSO estimator, we only need LASSO estimator that is obtained by a fast and stable optimization procedure such as LARS (Least Angle Regression) under LASSO modification or coordinate descent. And, the simplicity of our scaling method enables us to derive SURE (Stein's Unbiased Risk Estimate) under the modified LASSO estimator with scaling. Our scaling method together with model selection based on SURE is fully empirical and do not need additional hyper-parameters. In a simple numerical example, we verified that our scaling method actually improves LASSO and the SURE based model selection criterion can stably choose an appropriate sparse model.

In [58]:
dist, idx = rec.kneighbors(data.loc[paper.index, node2vec_cols].values.reshape(-1, len(node2vec_cols)), n_neighbors=5)
pd.DataFrame(
    {
        'Index': idx.ravel(),
        'Distance': dist.ravel()
    }
)

/opt/homebrew/Caskroom/miniforge/base/envs/graphml/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Index,Distance
0,49798,0.000000
1,33504,0.028014
2,49521,0.028994
3,6726,0.029053
4,35854,0.029207


In [59]:
for i, ii in enumerate(idx.ravel()):
    if i == 0:
        print("Selected paper:")
    if i == 1:
        print(" -- Recommendations -- ")
    print(data.loc[ii, 'title'])

Selected paper:
On an improvement of LASSO by scaling
 -- Recommendations -- 
Deep Learning Algorithms for Signal Recognition in Long Perimeter Monitoring Distributed Fiber Optic Sensors
Learning Discriminative Hashing Codes for Cross-Modal Retrieval based on Multiorder Statistical Features
Parameter Estimation for the Stochastically Perturbed Navier-Stokes Equations
Two-stage Plant Species Recognition by Combining Local K-NN and Weighted Sparse Representation


## Using 10 Dimensions from UMAP

In [60]:
rec = NearestNeighbors(n_neighbors=20)
rec.fit(data.loc[:, umap_cols])

NearestNeighbors(n_neighbors=20)

In [61]:
dist, idx = rec.kneighbors(data.loc[paper.index, umap_cols].values.reshape(-1, len(umap_cols)), n_neighbors=5)
pd.DataFrame(
    {
        'Index': idx.ravel(),
        'Distance': dist.ravel()
    }
)

/opt/homebrew/Caskroom/miniforge/base/envs/graphml/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Index,Distance
0,49798,0.000000
1,49903,0.141386
2,41519,0.199056
3,19802,0.286871
4,13285,0.417143


In [62]:
for i, ii in enumerate(idx.ravel()):
    if i == 0:
        print("Selected paper:")
    if i == 1:
        print(" -- Recommendations -- ")
    print(data.loc[ii, 'title'])

Selected paper:
On an improvement of LASSO by scaling
 -- Recommendations -- 
Bayesian Hypothesis Testing: Redux
A Parallel Best-Response Algorithm with Exact Line Search for Nonconvex Sparsity-Regularized Rank Minimization
Stochastic First- and Zeroth-order Methods for Nonconvex Stochastic Programming
Variance components and generalized Sobol' indices
